## Schema of our building
Inserer image

In [ ]:
[!bldg](builidng.pdf)

## Dimensions of the building

Here, we compute the dimensions of our 2 rooms:
we have chosen to take into account only one wall in contact with the rest of the chalet and one wall with the 2m² window in contact with the outside

In [ ]:
l_br=3 #lenght of the bedroom
L_br=4 #width of the bedroom
l_d= 3 #lenght of the dressing
L_d= 2 #width of the dressing
height= 3   #height under ceiling

S_br= l_br*L_br  #total surface bedroom
S_d= l_d*L_d    #total surface dressing

S_glass= 2   # surface of the glass taken into account in the study

S_wall_3=l_d*height  # surface of the wall between the dressing and the bedroom
S_wall_4=L_d*height  # surface of the wall of the dressing
S_wall_2= L_br*height # surface de droite 
S_wall_1= l_br*height - S_glass #surface du haut
 



h_in = 8 #convection coefficient for inside, W/m² K;
h_out= 25 #convection coefficient for outside, W/m² K;
Tin = 20    #temperature inside, K;
σ = σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant


## Desciption of the building
inserer photo adri

## Thermo-physical properties

In [ ]:
wood = {'Conductivity': 0.06,
            'Density': ,
            'Specific heat': ,
            'Width':0.2 ,
            'Surface': S_wall_1+S_wall_2+S_wall_3+S_wall_4
             }

whool = {'Conductivity': 0.034,
              'Density': ,
              'Specific heat': ,
              'Width':0.25 ,
              'Surface': S_wall_1+S_wall_2+S_wall_3+S_wall_4 }

glass = {'Conductivity': 1 ,
         'Density': ,
         'Specific heat': ,
         'Width':0.04 ,
         'Surface': S_glass}

air = {'Conductivity': 0.024 ,
         'Density': ,
         'Specific heat': ,
         'Width':0.1 ,
         'Surface': S_glass }   
          

## Radiative properties

In [ ]:
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

## Thermal circuit
Inserer photo


## Thermal conductances

### Conduction

In [ ]:
G_1 = wood['Conductivity'] / wood['Width'] * S_wall_1
G_2 = wood['Conductivity'] / wood['Width'] * S_wall_1
G_3 = whool['Conductivity'] / whool['Width'] * S_wall_1
G_4 = whool['Conductivity'] / whool['Width'] * S_wall_1
G_10= wood['Conductivity'] / wood['Width'] * S_wall_2
G_17= wood['Conductivity'] / wood['Width'] * S_wall_3
G_14= wood['Conductivity'] / wood['Width'] * S_wall_4
G_13= glass['Conductivity'] / glass['Width'] * S_glass
G_12= glass['Conductivity'] / glass['Width'] * S_glass

### Convection

In [ ]:
G_0 = h_out * S_wall_1
G_5= h_in * S_wall_1
G_11= h_in * S_wall_2
G_16= h_in * S_wall_3
G_15= h_in * S_wall_4


### Long wave radiation

In [ ]:
#Long Wave radiation
Tm = Tin + 273   # K, mean temp for radiative exchange
Fwg = glass['Surface']  / wood['Surface']    # view factor wall-glass

#Between window and wall_3
G7_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G7_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G7_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * glass['Surface']
G7 = 1 / (1 / G7_1 + 1 / G7_12 + 1 / G7_2) 

#Between wall_1 and wall_3
G8_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G8_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G8_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * S_wall_1 
G8 = 1 / (1 / G8_1 + 1 / G8_12 + 1 / G8_2)

## Thermal capacities

In [ ]:
C1 = wood['Density'] * wood['Specific heat'] * wood['Surface'] * wood['Width']      #wood capacity wall 1
C2 = whool['Density'] * whool['Specific heat'] * whool['Surface'] * whool['Width']  #whool capacity wall 1